In [6]:
import cv2
import numpy as np
from ultralytics import YOLO

VIDEO_PATH = 'Snooker2.mp4'

MODEL_NAME = 'yolo11s.pt'


CONF_THRESHOLD = 0.009

def main():

    print(f"Wczytuję model {MODEL_NAME}...")
    model = YOLO(MODEL_NAME)

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"Błąd: Nie można otworzyć wideo: {VIDEO_PATH}")
        return

    print("Naciśnij 'q', aby zakończyć.")

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        h, w, c = frame.shape


        margin_y = int(h * 0.15) # 15% z góry/dołu
        margin_x = int(w * 0.05) # 5% z boków

        roi_y_start = margin_y
        roi_y_end = h - margin_y
        roi_x_start = margin_x
        roi_x_end = w - margin_x

        cropped_frame = frame[roi_y_start:roi_y_end, roi_x_start:roi_x_end]

        if cropped_frame.size == 0:
            continue


        results = model(cropped_frame, conf=CONF_THRESHOLD, classes=[32], verbose=False)

        # Przygotowanie klatki do rysowania (oryginalnej)
        annotated_frame = frame.copy()

        # Rysujemy ramkę ROI dla podglądu (niebieska)
        cv2.rectangle(annotated_frame, (roi_x_start, roi_y_start), (roi_x_end, roi_y_end), (255, 0, 0), 2)


        for result in results:
            for box in result.boxes:

                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                confidence = float(box.conf[0])

                # Rozmiar wykrytego obiektu
                box_w = x2 - x1
                box_h = y2 - y1


                if box_w > 100 or box_h > 100:
                    continue


                if box_w < 5 or box_h < 5:
                    continue

                final_x1 = int(x1 + margin_x)
                final_y1 = int(y1 + margin_y)
                final_x2 = int(x2 + margin_x)
                final_y2 = int(y2 + margin_y)

                cv2.rectangle(annotated_frame, (final_x1, final_y1), (final_x2, final_y2), (0, 255, 0), 2)

                # Podpis z pewnością
                label = f"{confidence:.2f}"
                cv2.putText(annotated_frame, label, (final_x1, final_y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Wyświetlanie
        cv2.imshow('YOLO11 - Snooker Detection', annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

Wczytuję model yolo11s.pt...
Naciśnij 'q', aby zakończyć.
